## SEQ2Sql

Seq2sql implemented using BERT encoding instead of Glove.

### Import Libraries

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import json
from transformers import BertTokenizer, glue_convert_examples_to_features, BertModel
import numpy as np

### Import Preprocessed Dataset

Pre processed, tokenized WikiSql data is stored in './data/tokenized*'

In [4]:
dev_set_tokenized = pd.read_json('./data/tokenized_dev.jsonl',lines = True)
dev_set_tokenized.head()

,phase,table_id,question,sql,tokenized_query,query,tokenized_question
0,1,1-10015132-11,What position does the player who played for b...,"{'sel': 3, 'conds': [[5, 0, 'Butler CC (KS)']]...","[SELECT, position, FROM, table_, WHERE, school...",SELECT position FROM table_ WHERE school/club ...,"[what, position, does, the, player, who, playe..."
1,1,1-10015132-11,How many schools did player number 3 play at?,"{'sel': 5, 'conds': [[1, 0, '3']], 'agg': 3}","[SELECT, COUNT, (, school/club, team, ), FROM,...",SELECT COUNT ( school/club team ) FROM table_ ...,"[how, many, schools, did, player, number, 3, p..."
2,1,1-10015132-11,What school did player number 21 play for?,"{'sel': 5, 'conds': [[1, 0, '21']], 'agg': 0}","[SELECT, school/club, team, FROM, table_, WHER...",SELECT school/club team FROM table_ WHERE no ....,"[what, school, did, player, number, 21, play, ..."
3,1,1-10015132-11,Who is the player that wears number 42?,"{'sel': 0, 'conds': [[1, 0, '42']], 'agg': 0}","[SELECT, player, FROM, table_, WHERE, no, ., E...",SELECT player FROM table_ WHERE no . EQL 42,"[who, is, the, player, that, wears, number, 42..."
4,1,1-10015132-11,What player played guard for toronto in 1996-97?,"{'sel': 0, 'conds': [[3, 0, 'Guard'], [4, 0, '...","[SELECT, player, FROM, table_, WHERE, position...",SELECT player FROM table_ WHERE position EQL g...,"[what, player, played, guard, for, toronto, in..."


In [5]:
dev_set = pd.read_json('./data/dev.jsonl',lines = True)
dev_set.head()

,phase,table_id,question,sql
0,1,1-10015132-11,What position does the player who played for b...,"{'sel': 3, 'conds': [[5, 0, 'Butler CC (KS)']]..."
1,1,1-10015132-11,How many schools did player number 3 play at?,"{'sel': 5, 'conds': [[1, 0, '3']], 'agg': 3}"
2,1,1-10015132-11,What school did player number 21 play for?,"{'sel': 5, 'conds': [[1, 0, '21']], 'agg': 0}"
3,1,1-10015132-11,Who is the player that wears number 42?,"{'sel': 0, 'conds': [[1, 0, '42']], 'agg': 0}"
4,1,1-10015132-11,What player played guard for toronto in 1996-97?,"{'sel': 0, 'conds': [[3, 0, 'Guard'], [4, 0, '..."


In [6]:
dev_tables = pd.read_json('./data/dev.tables.jsonl',lines = True)
dev_tables.head()

,header,page_title,types,id,section_title,caption,rows,name,page_id
0,"[Player, No., Nationality, Position, Years in ...",Toronto Raptors all-time roster,"[text, text, text, text, text, text]",1-10015132-11,L,L,"[[Antonio Lang, 21, United States, Guard-Forwa...",table_10015132_11,NaN
1,"[Player, No., Nationality, Position, Years in ...",Toronto Raptors all-time roster,"[text, text, text, text, text, text]",1-10015132-9,J,J,"[[Jarrett Jack, 1, United States, Guard, 2009-...",table_10015132_9,NaN
2,[Entered office as Head of State or Government...,List of longest-serving G8 leaders,"[text, text, text, text, text]",1-10026563-1,Chronology of longest serving G8 Leaders,Chronology of longest serving G8 Leaders,"[[16 October 1964, 15 November 1975, 5 April 1...",table_10026563_1,NaN
3,"[Country ( exonym ), Capital ( exonym ), Count...",NaN,"[text, text, text, text, text]",1-1008653-1,NaN,NaN,"[[Afghanistan, Kabul, Afghanestan افغانستان, K...",table_1008653_1,NaN
4,"[No. in set, No. in series, Title, Directed by...",NaN,"[real, real, text, text, text, text, text]",1-10088101-1,NaN,NaN,"[[1, 174, "" Per Manum "", Kim Manners, Chris Ca...",table_10088101_1,NaN


In [7]:
dev_tables_tokenized = pd.read_json('./data/tokenized_dev.tables.jsonl',lines = True)
dev_tables_tokenized.head()

,header,page_title,types,id,section_title,caption,rows,name,page_id,tokenized_header
0,"[Player, No., Nationality, Position, Years in ...",Toronto Raptors all-time roster,"[text, text, text, text, text, text]",1-10015132-11,L,L,"[[Antonio Lang, 21, United States, Guard-Forwa...",table_10015132_11,NaN,"[[player], [no, .], [nationality], [position],..."
1,"[Player, No., Nationality, Position, Years in ...",Toronto Raptors all-time roster,"[text, text, text, text, text, text]",1-10015132-9,J,J,"[[Jarrett Jack, 1, United States, Guard, 2009-...",table_10015132_9,NaN,"[[player], [no, .], [nationality], [position],..."
2,[Entered office as Head of State or Government...,List of longest-serving G8 leaders,"[text, text, text, text, text]",1-10026563-1,Chronology of longest serving G8 Leaders,Chronology of longest serving G8 Leaders,"[[16 October 1964, 15 November 1975, 5 April 1...",table_10026563_1,NaN,"[[entered, office, as, head, of, state, or, go..."
3,"[Country ( exonym ), Capital ( exonym ), Count...",None,"[text, text, text, text, text]",1-1008653-1,None,None,"[[Afghanistan, Kabul, Afghanestan افغانستان, K...",table_1008653_1,NaN,"[[country, (, exonym, )], [capital, (, exonym,..."
4,"[No. in set, No. in series, Title, Directed by...",None,"[real, real, text, text, text, text, text]",1-10088101-1,None,None,"[[1, 174, "" Per Manum "", Kim Manners, Chris Ca...",table_10088101_1,NaN,"[[no, ., in, set], [no, ., in, series], [title..."


Load the data into `sql_data` and `table_data`, the `table_data` needs to be a dictionary

In [8]:
sql_data = []
with open('./data/tokenized_dev.jsonl') as lines:
    for i,line in enumerate(lines):
        data = json.loads(line.strip())
        sql_data.append(data)

In [9]:
table_data = {}
with open('./data/tokenized_dev.tables.jsonl') as lines:
     for i, line in enumerate(lines):
        data =  json.loads(line.strip())
        table_data[data['id']] = data

In [10]:
print(len(table_data))

2716


#### Universal function for loading the data

By passing the value `train` `dev` or `test` load the training dataset, validation dataset or the testing dataset respecitvily

In [11]:
def load_data_set(dataset_name):
    """
    Universal function to load the dataset
    Input: dataset_name - str {train, test, dev}
    Output: (sql_data, table_data) - (list, dictionary): containg the tokenized queries and the table dataset.
    """
    print(f"Loading {dataset_name} dataset")
    sql_file = './data/tokenized_'+dataset_name+'.jsonl'
    tables_file = './data/tokenized_'+dataset_name+'.tables.jsonl'
    sql_query_count = 0
    table_count = 0
    sql_data = []
    table_data = {}
    
    with open(sql_file) as lines:
        for line in lines:
            sql = json.loads(line.strip()) #remove white space before and after 
            sql_data.append(sql)
            sql_query_count += 1
    with open(tables_file) as lines:
        for line in lines:
            tab = json.loads(line.strip())
            table_data[tab['id']] = tab
            table_count += 1
    print(f"Loaded {sql_query_count} queries and {table_count} tables")
    return sql_data,table_data

In [12]:
train_sql, train_table = load_data_set('train')
print(f"{train_sql[0]}\n{train_table[train_sql[0]['table_id']]}")

Loading train dataset
Loaded 56355 queries and 18585 tables
{'phase': 1, 'table_id': '1-1000181-1', 'question': 'Tell me what the notes are for South Australia ', 'sql': {'sel': 5, 'conds': [[3, 0, 'SOUTH AUSTRALIA']], 'agg': 0}, 'tokenized_query': ['SELECT', 'notes', 'FROM', 'table_', 'WHERE', 'current', 'slogan', 'EQL', 'south', 'australia'], 'query': 'SELECT notes FROM table_ WHERE current slogan EQL south australia', 'tokenized_question': ['tell', 'me', 'what', 'the', 'notes', 'are', 'for', 'south', 'australia']}
{'id': '1-1000181-1', 'header': ['State/territory', 'Text/background colour', 'Format', 'Current slogan', 'Current series', 'Notes'], 'types': ['text', 'text', 'text', 'text', 'text', 'text'], 'rows': [['Australian Capital Territory', 'blue/white', 'Yaa·nna', 'ACT · CELEBRATION OF A CENTURY 2013', 'YIL·00A', 'Slogan screenprinted on plate'], ['New South Wales', 'black/yellow', 'aa·nn·aa', 'NEW SOUTH WALES', 'BX·99·HI', 'No slogan on current series'], ['New South Wales', 'b

In [13]:
tuple(x for x in train_sql[0]['sql']['conds'])
#val_sql[0]['sql']['sel']

([3, 0, 'SOUTH AUSTRALIA'],)

#### Function to create batch sequences.

Given a batch size this function creates batch sequences.  
The required indecies are passsed to idxes.

(This method of loading data is discouraged and need to implement `torch.utils.data.Dataset` and `Dataloader`)

In [14]:
def gen_batch_sequence(sql_data, table_data,idxes,start, end):
    # [3,5,7,8]
    question_seq = []
    column_seq = []
    number_of_col = []
    answer_seq = []
    query_seq = []
    ground_truth_cond_seq = []
    raw_data = []
    for i in range(start, end):
        sql = sql_data[idxes[i]]
        table_id = sql['table_id']
        question_seq.append(sql['tokenized_question'])
        column_seq.append(table_data[table_id]['tokenized_header'])
        number_of_col.append(len(table_data[table_id]['header']))
        answer_seq.append((sql['sql']['agg'],
                           sql['sql']['sel'],
                           len(sql['sql']['conds']),
                           tuple(x[0] for x in sql['sql']['conds']),
                            tuple(x[1] for x in sql['sql']['conds'])
                          ))
        query_seq.append(sql['tokenized_query'])
        ground_truth_cond_seq.append(sql['sql']['conds'])
        raw_data.append((sql['question'],table_data[table_id]['header'], sql['query']))
        
    return (question_seq, column_seq, number_of_col, answer_seq, query_seq, ground_truth_cond_seq,raw_data)

-----


#### Import BERT tokenizer 

In [15]:
PRE_TRAINED_MODEL_NAME = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME)
bertModel = BertModel.from_pretrained(PRE_TRAINED_MODEL_NAME)

# Freeze the bertModel
for params in bertModel.parameters():
    params.requires_grad = False

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [16]:
q_seq,col_seq, num_col, answer_seq, query_seq,gt_cond,rd = gen_batch_sequence(train_sql,train_table,[23,4,654],0,3)

In [17]:
col_seq[0]

[['player'],
 ['no', '.'],
 ['nationality'],
 ['position'],
 ['years', 'in', 'toronto'],
 ['school/club', 'team']]

In [18]:
for i, (one_q, one_col) in enumerate(zip(q_seq, col_seq)):
    #print(f"{i}: {one_q} || {one_col}")
    one_col_all = [x for toks in one_col for x in toks + ['[SEP]'] ]
    print(one_col_all)

['player', '[SEP]', 'no', '.', '[SEP]', 'nationality', '[SEP]', 'position', '[SEP]', 'years', 'in', 'toronto', '[SEP]', 'school/club', 'team', '[SEP]']
['order', 'year', '[SEP]', 'manufacturer', '[SEP]', 'model', '[SEP]', 'fleet', 'series', '(', 'quantity', ')', '[SEP]', 'powertrain', '(', 'engine/transmission', ')', '[SEP]', 'fuel', 'propulsion', '[SEP]']
['#', '[SEP]', 'episode', '[SEP]', 'air', 'date', '[SEP]', 'rating', '[SEP]', 'share', '[SEP]', '18–49', '(', 'rating/share', ')', '[SEP]', 'viewers', '(', 'm', ')', '[SEP]', 'weekly', 'rank', '[SEP]']


In [19]:
text = ['[SEP]']
encode = tokenizer.encode_plus(text=text,add_special_tokens = True,
                                #truncation=True,
                                return_token_type_ids = True,
                                padding = 'longest', 
                                return_attention_mask = True,
                                return_tensors = 'pt')
encode

{'input_ids': tensor([[101, 102, 102]]), 'token_type_ids': tensor([[0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1]])}

In [20]:
print(q_seq)
batch_queries = [' '.join(x) for x in q_seq]
q_encode = tokenizer.batch_encode_plus(batch_text_or_text_pairs=batch_queries ,
                                #max_length = 60,
                                add_special_tokens = True,
                                #truncation=True,
                                return_token_type_ids = True,
                                padding = 'longest', 
                                return_attention_mask = True,
                                return_tensors = 'pt')
bert_op = bertModel(**q_encode)
print(type(bert_op))
#print(q_encode["input_ids"])

[['what', 'number', 'is', 'the', 'player', 'that', 'played', '1998-2001'], ['how', 'many', 'times', 'is', 'the', 'fuel', 'propulsion', 'is', 'cng', '?'], ['what', 'is', 'the', 'episode', 'where', '18-49', 'has', 'a', 'rating/share', 'of', '3.5/9']]
<class 'transformers.modeling_outputs.BaseModelOutputWithPoolingAndCrossAttentions'>


In [21]:
print(bert_op.keys())

odict_keys(['last_hidden_state', 'pooler_output'])


In [22]:
'''
Converting This: [['player'],
 ['no', '.'],
 ['nationality'],
 ['position'],
 ['years', 'in', 'toronto'],
 ['school/club', 'team']]
 
 Into : player | no . | nationality | position | years in toronto | school/club team |
 
 for each element in the batch
'''

separator = '[SEP]'
header_batch_list=list(
                   map(lambda col: ' '.join([x for tok in col for x in tok + [separator]]), col_seq)
                    )
print(header_batch_list[0])

# for col in col_seq:
#     print(' '.join([x for tok in col for x in tok + [separator]]))
col_encode = tokenizer.batch_encode_plus(batch_text_or_text_pairs= header_batch_list,
                                 # max_length = 60,
                                add_special_tokens = True,
                                #truncation=True,
                                return_token_type_ids = True,
                                padding = 'longest', 
                                return_attention_mask = True,
                                return_tensors = 'pt')
col_encode['input_ids']

player [SEP] no . [SEP] nationality [SEP] position [SEP] years in toronto [SEP] school/club team [SEP]


tensor([[  101,  2447,   102,  2053,  1012,   102, 10662,   102,  2597,   102,
          2086,  1999,  4361,   102,  2082,  1013,  2252,  2136,   102,   102,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0],
        [  101,  2344,  2095,   102,  7751,   102,  2944,   102,  4170,  2186,
          1006, 11712,  1007,   102,  2373, 23654,  1006,  3194,  1013,  6726,
          1007,   102,  4762, 16404,   102,   102,     0,     0,     0,     0],
        [  101,  1001,   102,  2792,   102,  2250,  3058,   102,  5790,   102,
          3745,   102,  2324,  1516,  4749,  1006,  5790,  1013,  3745,  1007,
           102,  7193,  1006,  1049,  1007,   102,  4882,  4635,   102,   102]])

In [23]:
'''
Converting This: [['player'],
 ['no', '.'],
 ['nationality'],
 ['position'],
 ['years', 'in', 'toronto'],
 ['school/club', 'team']]
 
 Into : player [SEP] no . [SEP] nationality [SEP] position [SEP] years in toronto [SEP] school/club team [SEP]
 
 for each element in the batch
 
 and combining the question with the above table header
 Eg:
 ['what', 'number', 'is', 'the', 'player', 'that', 'played', '1998-2001']
     and
 ['player', '[SEP]', 'no', '.', '[SEP]', 'nationality', '[SEP]', 'position', '[SEP]', 'years', 'in', 'toronto', '[SEP]', 'school/club', 'team', '[SEP]']
 
 into: what number is the player that played 1998-2001 [SEP] player [SEP] no . [SEP] nationality [SEP] position [SEP] years in toronto [SEP] school/club team [SEP]
'''

separator = '[SEP]'
header_batch_list=list(
                   map(lambda col: [x for tok in col for x in tok + [separator]], col_seq)
                    )
# print(header_batch_list[0])
# print(q_seq)

input_string = []
for i in range(len(header_batch_list)):
    input_string.append(' '.join(q_seq[i] +[separator]+ header_batch_list[i][:-1]))

print(input_string[0])    
tokenizer_args = { 'add_special_tokens' : True,
                   'return_token_type_ids' : True,
                   'padding' : 'longest', 
                   'return_attention_mask' : True,
                   'return_tensors' : 'pt'}
# for col in col_seq:
#     print(' '.join([x for tok in col for x in tok + [separator]]))
col_encode = tokenizer.batch_encode_plus(batch_text_or_text_pairs= input_string, **tokenizer_args)
col_encode['input_ids']

what number is the player that played 1998-2001 [SEP] player [SEP] no . [SEP] nationality [SEP] position [SEP] years in toronto [SEP] school/club team


tensor([[  101,  2054,  2193,  2003,  1996,  2447,  2008,  2209,  2687,  1011,
          2541,   102,  2447,   102,  2053,  1012,   102, 10662,   102,  2597,
           102,  2086,  1999,  4361,   102,  2082,  1013,  2252,  2136,   102,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0],
        [  101,  2129,  2116,  2335,  2003,  1996,  4762, 16404,  2003, 27166,
          2290,  1029,   102,  2344,  2095,   102,  7751,   102,  2944,   102,
          4170,  2186,  1006, 11712,  1007,   102,  2373, 23654,  1006,  3194,
          1013,  6726,  1007,   102,  4762, 16404,   102,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0],
        [  101,  2054,  2003,  1996,  2792,  2073,  2324,  1011,  4749,  2038,
          1037,  5790,  1013,  3745,  1997,  1017,  1012,  1019,  1013,  1023,
           102,  1001,   102,  2792,   102,  2250,  3058,   102,

In [24]:
col_encode['input_ids'].shape

torch.Size([3, 49])

In [25]:
bert_op = bertModel(**col_encode)


In [40]:
print(bert_op.last_hidden_state[0])
bert_op.last_hidden_state[0].shape

tensor([[-0.7509, -0.0451, -0.3663,  ..., -0.3276,  0.1047,  0.8885],
        [-0.4485,  0.3220, -0.4430,  ...,  0.2473,  0.1332, -0.1755],
        [ 0.1347, -0.5952,  0.0330,  ..., -0.5421,  0.0289,  0.8840],
        ...,
        [-0.6576,  0.1654, -0.0034,  ..., -0.1518, -0.1862,  0.2657],
        [-0.6042,  0.2060, -0.0119,  ...,  0.0314,  0.0453,  0.3622],
        [-0.3505,  0.1392, -0.1125,  ..., -0.2700, -0.1290,  0.4474]])


torch.Size([49, 768])

In [27]:
print(list(col_encode.keys()))

['input_ids', 'token_type_ids', 'attention_mask']


In [28]:
x = torch.randn(2,3)
y = torch.randn(2,4)
z = torch.cat((x,y), dim=1)
print(f"X: {x}\nY: {y}\nZ: {z}")

X: tensor([[-0.1272,  1.5787, -0.4264],
        [-0.7108,  1.7789, -0.0619]])
Y: tensor([[-2.1488, -0.1765, -0.2232,  0.1911],
        [-0.8833,  1.1074,  1.6308,  0.7582]])
Z: tensor([[-0.1272,  1.5787, -0.4264, -2.1488, -0.1765, -0.2232,  0.1911],
        [-0.7108,  1.7789, -0.0619, -0.8833,  1.1074,  1.6308,  0.7582]])


In [29]:
con_cat_string = torch.cat((q_encode['input_ids'],col_encode['input_ids']),dim = 1)
print(con_cat_string)

tensor([[  101,  2054,  2193,  2003,  1996,  2447,  2008,  2209,  2687,  1011,
          2541,   102,     0,     0,     0,     0,     0,     0,     0,     0,
             0,   101,  2054,  2193,  2003,  1996,  2447,  2008,  2209,  2687,
          1011,  2541,   102,  2447,   102,  2053,  1012,   102, 10662,   102,
          2597,   102,  2086,  1999,  4361,   102,  2082,  1013,  2252,  2136,
           102,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0],
        [  101,  2129,  2116,  2335,  2003,  1996,  4762, 16404,  2003, 27166,
          2290,  1029,   102,     0,     0,     0,     0,     0,     0,     0,
             0,   101,  2129,  2116,  2335,  2003,  1996,  4762, 16404,  2003,
         27166,  2290,  1029,   102,  2344,  2095,   102,  7751,   102,  2944,
           102,  4170,  2186,  1006, 11712,  1007,   102,  2373, 23654,  1006,
          3194,  1013,  6726,  1007,   102,  4762, 

In [30]:
tokenizer.convert_ids_to_tokens(col_encode['input_ids'][0])

['[CLS]',
 'what',
 'number',
 'is',
 'the',
 'player',
 'that',
 'played',
 '1998',
 '-',
 '2001',
 '[SEP]',
 'player',
 '[SEP]',
 'no',
 '.',
 '[SEP]',
 'nationality',
 '[SEP]',
 'position',
 '[SEP]',
 'years',
 'in',
 'toronto',
 '[SEP]',
 'school',
 '/',
 'club',
 'team',
 '[SEP]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]']

In [31]:
## function to train the model
def epoch_train(model, optim, batch_size, sql_data, table_data):
    model.train()

## WordEmbedding Module

In [32]:
class WordEmbedding(nn.Module):
    def __init__(self, bert_encoder, max_length=None, separator = '[SEP]'):
        super(WordEmbedding, self).__init__()
        self.bert_tokenizer = BertTokenizer.from_pretrained(bert_encoder)
        self.bert_model = BertModel.from_pretrained(bert_encoder)
        self.max_length = max_length
        self.gpu = torch.cuda.is_available()
        self.seperator = separator
        
        self.bert_args = { 'add_special_tokens' : True,
                           'return_token_type_ids' : True,
                           'padding' : 'longest', 
                           'return_attention_mask' : True,
                           'return_tensors' : 'pt'}
        
        if(self.max_length != None):
            bert_args['max_length'] = self.max_length
            bert_args['padding'] = 'max_length'
        for param in self.bert_model.parameters():
            param.requires_grad = False
        assert list(self.bert_model.parameters())[0].requires_grad == False 
        
        if self.gpu:
            self.bert_model = self.bert_model.to('cuda')
    
    def gen_x_batch(self,q_batch,col_batch):
        '''
        Input: q_batch: list of tokenized query string i.e. List[List].
               col_batch: list of tokenzed header of the corresponding table header. List[List[List]]
        Output: ret_dict: a dictionary containing the 
        '''
        batch_queries = [' '.join(x) for x in q_batch]
        if batch_queries == []:
            print(q_batch, col_batch)
        '-------'
        header_batch_list=list(
                   map(lambda col: [x for tok in col for x in tok + [separator]], col_batch)
                    )

        input_string_list = []
        for i in range(len(header_batch_list)):
            input_string_list.append(' '.join(q_batch[i] +[separator]+ header_batch_list[i][:-1]))

        '-------'
        inp_encode = self.bert_tokenizer.batch_encode_plus(batch_text_or_text_pairs=input_string_list ,**self.bert_args)
        if self.gpu:
            for key in inp_encode.keys():
                inp_encode[key] = inp_encode[key].to('cuda')
        bert_op = self.bert_model(**inp_encode)
        
        # odict_keys(['last_hidden_state', 'pooler_output'])    
        return bert_op
        

In [33]:
#train_sql, train_table = load_data_set(data_set)
#def test_wordembed_module(data_set, batch_size = 32):
def test_wordembed_module(train_sql,train_table,batch_size = 32): 
    word_emb = WordEmbedding('bert-base-uncased')
    start = 0
    end = batch_size
    num_x = len(train_sql)
    idxes = np.random.permutation(num_x)
    i = 0
    while start < num_x:
        end = start+batch_size if start+batch_size <= num_x else num_x
        ret_tuple = gen_batch_sequence(train_sql,train_table,idxes, start, end)
        if ret_tuple[0] == []:
            print(start, end)
        bert_op = word_emb.gen_x_batch(ret_tuple[0],ret_tuple[1])
        last_hidden_state,pooler_output = bert_op.last_hidden_state, bert_op.pooler_output 
        
        assert last_hidden_state.size()[2] == 768
        assert last_hidden_state.size()[0] <= batch_size
        assert pooler_output.size()[1] == 768
        assert pooler_output.size()[0] <= batch_size
        
        if (i%10 == 0):
            print(f"\ni={i}\nSanity Check!!\n")
            print(f"Start: {start}\nEnd: {end}\n")
            print(f"Size of LastHidden Layer Size:{last_hidden_state.size()}\nPooler output size:{pooler_output.size()}\n\
            LastHidden Layer: {last_hidden_state}\nPooler output: {pooler_output}\n")
            
        i = i+1
        start = end
    
test_wordembed_module(train_sql,train_table)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).



i=0
Sanity Check!!

Start: 0
End: 32

Size of LastHidden Layer Size:torch.Size([32, 91, 768])
Pooler output size:torch.Size([32, 768])
            LastHidden Layer: tensor([[[-0.1757,  0.1176, -0.0385,  ..., -0.3337,  0.2855,  0.7696],
         [-0.1743, -0.1953, -0.2888,  ...,  0.3705,  0.0293, -0.4341],
         [-0.2986, -0.3153, -0.0748,  ..., -0.0767, -0.0712,  0.3809],
         ...,
         [-0.1913, -0.2420, -0.2525,  ...,  0.0772,  0.4887,  0.3546],
         [-0.1194, -0.1350, -0.3206,  ...,  0.0404,  0.4210,  0.3224],
         [-0.3075,  0.2801, -0.0397,  ..., -0.1871, -0.0273,  0.3829]],

        [[-0.8465, -0.1847, -0.0270,  ..., -0.2296,  0.4143,  0.7087],
         [-0.9341, -0.2153,  0.6237,  ..., -0.1480,  0.5412, -0.0200],
         [-0.4342,  0.2370, -0.1530,  ..., -0.6994, -0.3456,  0.2828],
         ...,
         [-0.6433, -0.2636,  0.1400,  ..., -0.1164,  0.1008,  0.3192],
         [-0.5058, -0.1829,  0.0858,  ...,  0.0956,  0.2623, -0.1172],
         [-0.6582,  0.5


i=30
Sanity Check!!

Start: 960
End: 992

Size of LastHidden Layer Size:torch.Size([32, 64, 768])
Pooler output size:torch.Size([32, 768])
            LastHidden Layer: tensor([[[-6.5032e-01,  8.3251e-02, -3.5419e-02,  ..., -4.3575e-01,
           1.6364e-01,  6.8620e-01],
         [-2.2287e-01,  7.1706e-02, -1.4899e-01,  ...,  3.5431e-01,
           4.7144e-01, -8.8550e-02],
         [-7.6863e-01, -3.5141e-01, -1.3466e-01,  ...,  2.1026e-01,
          -2.2978e-01,  3.8933e-01],
         ...,
         [-6.5899e-02,  1.3975e-01,  3.2272e-01,  ..., -3.4987e-01,
           8.9843e-02, -6.9096e-02],
         [-2.0162e-01,  8.0221e-02, -4.2225e-01,  ...,  1.2996e-01,
           3.6398e-01, -4.8759e-02],
         [-1.5393e-01,  5.0784e-01, -1.8692e-01,  ..., -1.8612e-01,
           1.1236e-01, -1.8552e-02]],

        [[-3.6107e-01, -6.4294e-01,  4.0478e-01,  ..., -6.3596e-01,
           4.5443e-01,  6.3289e-01],
         [ 1.7928e-01, -1.2804e-02, -3.0635e-01,  ...,  2.3680e-01,
           


i=50
Sanity Check!!

Start: 1600
End: 1632

Size of LastHidden Layer Size:torch.Size([32, 88, 768])
Pooler output size:torch.Size([32, 768])
            LastHidden Layer: tensor([[[-3.9204e-01,  1.5092e-01, -2.7518e-02,  ..., -2.7190e-01,
           2.1748e-01,  9.4026e-01],
         [-2.3785e-01,  1.0356e-01,  1.5788e-01,  ...,  3.2609e-01,
           1.3817e-01, -1.3266e-01],
         [ 3.0454e-01, -4.5316e-01, -8.7580e-02,  ..., -5.7605e-01,
           3.1317e-01,  5.3209e-01],
         ...,
         [-1.8486e-01,  1.7464e-01,  1.4626e-01,  ..., -1.0801e-01,
           9.3258e-02,  3.6555e-01],
         [-1.7714e-01,  1.4709e-01,  1.3602e-01,  ..., -1.6471e-01,
           8.1176e-02,  2.3896e-01],
         [-1.2492e-01,  4.6101e-03,  2.9193e-02,  ..., -2.4863e-01,
           8.2629e-02,  5.5214e-01]],

        [[-9.4299e-01, -1.0643e-01, -7.6496e-02,  ..., -2.4270e-01,
           2.4956e-01,  7.2425e-01],
         [-3.7517e-02,  2.4295e-01, -4.0724e-01,  ...,  2.4250e-01,
         


i=80
Sanity Check!!

Start: 2560
End: 2592

Size of LastHidden Layer Size:torch.Size([32, 61, 768])
Pooler output size:torch.Size([32, 768])
            LastHidden Layer: tensor([[[-3.9861e-01,  4.1456e-02, -6.0846e-02,  ..., -2.9052e-01,
           1.2840e-01,  5.6053e-01],
         [-4.1043e-01, -2.1747e-01,  4.9763e-01,  ...,  3.0608e-01,
           3.3339e-01, -2.1527e-01],
         [-7.0603e-01, -3.0564e-01,  3.3954e-01,  ...,  2.3865e-02,
           2.8492e-01,  3.3936e-01],
         ...,
         [-2.8855e-01, -5.4686e-02,  3.7506e-02,  ..., -7.3083e-02,
           1.7431e-01,  3.2567e-01],
         [-7.8247e-02, -1.1127e-01,  2.1244e-01,  ..., -1.2989e-02,
           1.7113e-01, -4.4578e-03],
         [-1.3658e-01,  1.4455e-01,  6.4740e-01,  ...,  1.9113e-01,
           2.0852e-01, -1.7026e-01]],

        [[-7.1321e-01,  2.7847e-02, -1.6643e-01,  ..., -5.4106e-01,
           1.0992e-01,  7.1278e-01],
         [-5.0586e-01, -2.9268e-01, -7.9197e-02,  ..., -5.4014e-02,
         

...


i=1630
Sanity Check!!

Start: 52160
End: 52192

Size of LastHidden Layer Size:torch.Size([32, 82, 768])
Pooler output size:torch.Size([32, 768])
            LastHidden Layer: tensor([[[-3.7820e-01, -3.8736e-01, -2.4625e-01,  ..., -4.6982e-01,
           1.7771e-02,  7.8919e-01],
         [-2.8635e-01, -4.1686e-01, -1.7904e-01,  ...,  5.4352e-01,
          -2.4891e-02, -1.1368e-01],
         [-3.7061e-01, -8.9633e-01,  1.7913e-01,  ...,  3.7755e-01,
          -6.6888e-01,  4.2602e-01],
         ...,
         [-4.2918e-02, -1.3481e-01,  1.5751e-01,  ..., -1.9536e-01,
          -3.6656e-01,  2.1199e-01],
         [-5.1297e-03, -2.8631e-01,  3.0772e-01,  ..., -1.4717e-01,
          -3.2365e-01,  5.4629e-02],
         [-9.0575e-02, -5.0921e-01, -4.0664e-01,  ...,  5.9931e-03,
          -1.0875e-01,  1.2801e-01]],

        [[-6.3423e-01, -2.0048e-01,  4.8541e-02,  ..., -5.4413e-01,
           3.6800e-01,  6.5813e-01],
         [-3.4322e-01, -7.0865e-02, -5.3570e-01,  ..., -5.4789e-02,
     


i=1660
Sanity Check!!

Start: 53120
End: 53152

Size of LastHidden Layer Size:torch.Size([32, 60, 768])
Pooler output size:torch.Size([32, 768])
            LastHidden Layer: tensor([[[-4.3045e-01, -1.3224e-01,  1.5999e-02,  ...,  8.1874e-02,
           9.9387e-02,  6.8630e-01],
         [ 3.3749e-01,  2.7138e-01, -6.0015e-01,  ...,  1.2368e-01,
           7.9495e-01,  1.9569e-01],
         [-5.7073e-01, -2.2291e-01, -5.8471e-01,  ...,  7.7933e-02,
           2.7576e-01,  3.2575e-02],
         ...,
         [-4.9287e-01, -1.8175e-01, -3.6304e-01,  ...,  4.0978e-01,
           2.2334e-01,  1.4097e-01],
         [-1.7148e-01, -1.7890e-01, -5.2890e-02,  ..., -5.1425e-02,
           1.2072e-01,  4.3270e-01],
         [-5.0337e-01, -5.9548e-02, -3.4959e-01,  ...,  4.1006e-01,
           2.7307e-01,  1.5290e-01]],

        [[-5.8260e-01,  4.0926e-01,  8.8262e-02,  ..., -2.3078e-01,
          -2.0260e-02,  6.1611e-01],
         [-4.5830e-01, -6.7089e-04, -1.2193e-01,  ...,  2.9247e-01,
     


i=1690
Sanity Check!!

Start: 54080
End: 54112

Size of LastHidden Layer Size:torch.Size([32, 63, 768])
Pooler output size:torch.Size([32, 768])
            LastHidden Layer: tensor([[[-0.5406,  0.1907,  0.0799,  ..., -0.1729,  0.2667,  0.6248],
         [ 0.0793,  0.1356, -0.3039,  ...,  0.5282,  0.7393, -0.1243],
         [-0.2988, -0.6647, -0.0056,  ...,  0.4174, -0.3269,  0.6095],
         ...,
         [-0.7627,  0.0322, -0.4370,  ...,  0.0810,  0.3819,  0.1935],
         [-0.5483,  0.1538,  0.0884,  ..., -0.0130,  0.1160,  0.0330],
         [-0.6646,  0.1582, -0.0832,  ...,  0.0304,  0.3399,  0.1292]],

        [[-0.6381, -0.4934, -0.0599,  ..., -0.6137,  0.0619,  0.6873],
         [-1.1445,  0.0095, -0.2501,  ..., -0.2385,  1.2918,  0.0570],
         [-0.4971, -1.3105, -0.0801,  ..., -0.2207, -0.3179,  0.5722],
         ...,
         [-0.4495, -0.8509,  0.5800,  ..., -0.0156,  0.0924,  0.3013],
         [-0.4419, -0.8851,  0.5872,  ..., -0.1594,  0.0085,  0.3560],
         [-0.


i=1720
Sanity Check!!

Start: 55040
End: 55072

Size of LastHidden Layer Size:torch.Size([32, 53, 768])
Pooler output size:torch.Size([32, 768])
            LastHidden Layer: tensor([[[-6.4273e-01,  9.8706e-02,  1.6834e-01,  ..., -4.1928e-01,
           7.8924e-02,  5.5039e-01],
         [-4.5838e-01,  2.9079e-02, -3.8853e-01,  ..., -3.1082e-02,
           2.4599e-01, -3.9891e-01],
         [-8.6715e-01, -2.4406e-01, -4.7569e-01,  ...,  5.7921e-02,
          -8.4018e-01,  2.1257e-01],
         ...,
         [-5.6851e-01, -5.4883e-02,  9.3904e-02,  ..., -8.1067e-02,
           2.5551e-01,  3.6979e-01],
         [-4.8647e-01, -2.9806e-02,  2.3521e-01,  ..., -2.4283e-01,
          -2.3816e-01,  2.5221e-01],
         [-6.2145e-01, -1.6952e-02,  1.8345e-01,  ..., -7.2951e-02,
           1.1519e-01,  2.5187e-01]],

        [[-3.1976e-01,  1.6316e-02,  5.4453e-02,  ..., -1.4589e-01,
           1.3574e-01,  5.3316e-01],
         [-2.9432e-01, -1.4468e-01, -4.4212e-01,  ...,  1.8994e-01,
     


i=1750
Sanity Check!!

Start: 56000
End: 56032

Size of LastHidden Layer Size:torch.Size([32, 107, 768])
Pooler output size:torch.Size([32, 768])
            LastHidden Layer: tensor([[[-6.5385e-01,  3.7598e-01,  8.0945e-02,  ..., -6.0115e-01,
           3.9073e-01,  6.4526e-01],
         [-2.7967e-01,  1.0394e-01, -2.5296e-01,  ...,  3.1094e-01,
           6.3427e-01, -8.7004e-02],
         [-3.3243e-01, -1.5565e-01,  5.6576e-02,  ..., -1.7243e-01,
          -1.8934e-01,  7.7372e-01],
         ...,
         [-4.0225e-01,  1.5062e-01,  3.6517e-01,  ..., -2.0617e-01,
           1.4784e-01,  3.4815e-01],
         [-5.1598e-01, -9.0487e-02,  3.9711e-02,  ...,  5.2704e-02,
           5.1585e-01,  5.5409e-01],
         [-5.9812e-01, -9.3608e-02,  1.1047e-02,  ..., -5.7418e-02,
           4.5501e-01,  4.5921e-01]],

        [[-6.4023e-01,  7.2433e-03,  1.6474e-02,  ..., -5.3677e-01,
           1.5203e-01,  5.6179e-01],
         [-1.0615e+00, -2.3915e-01,  5.1171e-02,  ..., -2.1272e-01,
    

In [1]:
from utils.utils import load_data_set
train_sql, train_table = load_data_set('train')

Loading train dataset
Loaded 56355 queries and 18585 tables


In [2]:
from model.wordEmbedding import test_wordembed_module

test_wordembed_module(train_sql, train_table)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).



i=0
Sanity Check!!

Start: 0
End: 32

Size of LastHidden Layer Size:torch.Size([32, 60, 768])
Pooler output size:torch.Size([32, 768])
            LastHidden Layer: tensor([[[-0.7360, -0.3183, -0.1671,  ..., -0.0541,  0.3680,  0.6488],
         [ 0.1146,  0.1187, -0.3189,  ...,  0.4162,  0.3891,  0.2190],
         [-0.8834, -0.8451,  0.4053,  ...,  0.1687, -0.0132,  0.3803],
         ...,
         [ 0.2565, -0.5440,  0.3062,  ...,  0.2203, -0.1344, -0.6219],
         [-0.0052,  0.3801,  0.2257,  ...,  0.0346, -0.6065,  0.1337],
         [ 0.8852,  0.0332, -0.2031,  ...,  0.2007, -0.5840, -0.3072]],

        [[-0.5216,  0.2069, -0.3126,  ..., -0.2208, -0.1085,  0.4717],
         [-0.5596,  0.3364, -0.4079,  ...,  0.3152,  0.2703, -0.0337],
         [-0.4433, -0.4492, -0.2297,  ...,  0.1253, -0.3978,  0.7244],
         ...,
         [-0.5026,  0.2304,  0.1200,  ...,  0.0251, -0.4091,  0.3194],
         [-0.6771, -0.0647,  0.2924,  ...,  0.2989, -0.4582,  0.5327],
         [-0.5909,  0.2


i=30
Sanity Check!!

Start: 960
End: 992

Size of LastHidden Layer Size:torch.Size([32, 60, 768])
Pooler output size:torch.Size([32, 768])
            LastHidden Layer: tensor([[[-0.3537, -0.4447,  0.0796,  ..., -0.6434,  0.1948,  0.8515],
         [-0.9852, -0.1915, -0.1561,  ..., -0.1698,  0.9630, -0.2320],
         [ 0.8798,  0.1429,  0.6082,  ..., -1.0878,  0.5556,  0.3765],
         ...,
         [-0.1552, -0.3071,  0.3599,  ..., -0.3503,  0.1202,  0.3466],
         [-0.3092, -0.4811,  0.2938,  ..., -0.2282,  0.2688,  0.3767],
         [ 0.0138, -0.3922,  0.3843,  ..., -0.2030,  0.1247,  0.4370]],

        [[-0.4347, -0.6686,  0.0100,  ..., -0.0589,  0.4466,  0.8045],
         [-0.5453, -1.2426, -0.6027,  ...,  0.2512, -0.0025, -0.6876],
         [-0.6054, -1.0845,  0.7297,  ..., -0.3916, -0.0925,  0.3934],
         ...,
         [-0.2433, -0.2602,  0.3958,  ...,  0.0854,  0.1304,  0.2709],
         [-0.3735, -0.4544,  0.4562,  ...,  0.0743,  0.3404,  0.3983],
         [-0.3145, 


i=60
Sanity Check!!

Start: 1920
End: 1952

Size of LastHidden Layer Size:torch.Size([32, 56, 768])
Pooler output size:torch.Size([32, 768])
            LastHidden Layer: tensor([[[-0.4186, -0.0572,  0.0276,  ...,  0.1255,  0.0612,  0.7230],
         [ 0.3679,  0.5419, -0.6098,  ...,  0.1080,  0.7428,  0.3347],
         [-0.6119,  0.4308, -0.9248,  ..., -0.0860,  0.3031,  0.1657],
         ...,
         [-0.5177, -0.1298, -0.5078,  ...,  0.4387,  0.2018,  0.2584],
         [-0.2327,  0.0586, -0.2301,  ..., -0.0572,  0.0420,  0.4678],
         [-0.2754, -0.0618, -0.3754,  ..., -0.0844,  0.0877,  0.5161]],

        [[-0.5108,  0.0236,  0.1160,  ..., -0.3224,  0.1250,  0.7706],
         [-0.5650,  0.1232, -0.3174,  ...,  0.5099,  0.2544, -0.2093],
         [-0.4858, -0.2105, -0.1697,  ...,  0.1047, -0.4189,  0.6615],
         ...,
         [-0.5259, -0.2331,  0.4895,  ..., -0.1473,  0.0127,  0.2249],
         [-0.5382, -0.2782,  0.4199,  ..., -0.0694,  0.0462,  0.1749],
         [-0.5001


i=90
Sanity Check!!

Start: 2880
End: 2912

Size of LastHidden Layer Size:torch.Size([32, 76, 768])
Pooler output size:torch.Size([32, 768])
            LastHidden Layer: tensor([[[-0.8624, -0.0187, -0.1579,  ..., -0.4395,  0.2138,  0.7921],
         [-0.0390,  0.2538, -0.2520,  ...,  0.2850,  0.4865,  0.3346],
         [-0.4315, -0.8027, -0.0808,  ..., -0.2743,  0.1257,  0.3221],
         ...,
         [-0.9073, -0.1401,  0.1069,  ...,  0.1155,  0.4105,  0.3667],
         [-0.3261,  0.1077, -0.0609,  ...,  0.0315, -0.2177,  0.5651],
         [-0.8243, -0.1224,  0.0633,  ...,  0.0428,  0.3990,  0.4129]],

        [[-0.7829, -0.0038, -0.1921,  ..., -0.5213,  0.0922,  0.8662],
         [ 0.1274, -0.1997, -0.2169,  ...,  0.1693,  0.4594, -0.1525],
         [-0.1129, -0.3617, -0.4215,  ..., -0.2722, -0.8605,  0.5087],
         ...,
         [-0.3882,  0.1002,  0.2162,  ..., -0.2754, -0.2753,  0.3368],
         [-0.3701,  0.1535,  0.1294,  ..., -0.1883, -0.2319,  0.3708],
         [-0.9301

...


i=1630
Sanity Check!!

Start: 52160
End: 52192

Size of LastHidden Layer Size:torch.Size([32, 61, 768])
Pooler output size:torch.Size([32, 768])
            LastHidden Layer: tensor([[[-6.0269e-01,  7.1275e-04,  4.6279e-02,  ..., -4.5669e-01,
           2.3427e-01,  6.5363e-01],
         [-5.0927e-01, -5.3110e-02, -1.3220e-01,  ..., -7.7408e-02,
           1.9144e-01, -4.2693e-01],
         [-5.1627e-01, -4.8831e-01,  1.3056e-01,  ..., -2.0895e-01,
          -5.4963e-01,  5.5507e-01],
         ...,
         [-3.9472e-01, -2.4220e-01,  4.0400e-01,  ..., -4.0355e-02,
          -6.7121e-02,  1.5441e-01],
         [-1.7485e-01,  2.4590e-03,  4.0965e-01,  ..., -4.1821e-02,
           9.4490e-02, -2.0762e-01],
         [-1.0930e-01, -1.2356e-02,  3.8546e-01,  ..., -3.1004e-02,
           1.2122e-01, -1.9480e-01]],

        [[-3.2275e-01, -5.1727e-01,  1.1121e-01,  ..., -5.9681e-01,
           3.2106e-01,  7.8569e-01],
         [ 2.2852e-01, -6.1362e-02, -9.7381e-02,  ..., -3.6339e-02,
     


i=1660
Sanity Check!!

Start: 53120
End: 53152

Size of LastHidden Layer Size:torch.Size([32, 85, 768])
Pooler output size:torch.Size([32, 768])
            LastHidden Layer: tensor([[[-4.3408e-01,  7.4592e-02, -6.4419e-02,  ..., -4.8164e-01,
           2.6781e-01,  5.5573e-01],
         [-3.9077e-01,  9.2923e-03, -1.6465e-01,  ..., -1.0893e-01,
           2.8301e-01, -5.8216e-01],
         [-1.9799e-01, -4.6346e-01,  1.5239e-01,  ..., -2.9148e-01,
          -4.7083e-01, -1.5661e-01],
         ...,
         [-8.2186e-01,  2.4676e-02, -2.9425e-01,  ..., -1.1999e-01,
           2.6301e-01,  3.9343e-01],
         [-8.6009e-01, -1.5682e-01, -3.4591e-01,  ..., -8.6983e-03,
           3.3206e-01,  3.1158e-01],
         [-8.9876e-01, -4.5643e-02, -4.0072e-01,  ..., -1.2384e-01,
           2.7741e-01,  3.0819e-01]],

        [[-8.7794e-01, -4.0558e-02, -7.0775e-02,  ..., -2.3561e-01,
           6.7390e-02,  7.3793e-01],
         [-1.0536e+00,  3.9628e-01,  5.4010e-02,  ...,  1.1054e-01,
     


i=1680
Sanity Check!!

Start: 53760
End: 53792

Size of LastHidden Layer Size:torch.Size([32, 56, 768])
Pooler output size:torch.Size([32, 768])
            LastHidden Layer: tensor([[[-0.6800,  0.1472, -0.0612,  ..., -0.0862,  0.0815,  0.5751],
         [-0.1573,  0.2973, -0.5203,  ...,  0.2956,  0.3587,  0.1714],
         [-0.7481, -0.0746,  0.3875,  ...,  0.3198, -0.0905, -0.3032],
         ...,
         [-0.7036,  0.0768, -0.4195,  ...,  0.0624,  0.0120,  0.4629],
         [-0.6132, -0.0554,  0.1923,  ...,  0.2306, -0.2806,  0.2096],
         [-0.7201,  0.1226, -0.3066,  ..., -0.0124, -0.0835,  0.4601]],

        [[-0.2427, -0.2362, -0.0333,  ..., -0.3565,  0.2436,  0.5166],
         [ 0.1475,  0.0067, -0.2569,  ...,  0.0457,  0.3437,  0.1695],
         [-0.1664, -0.6126,  0.0745,  ...,  0.0389,  0.1448, -0.3017],
         ...,
         [-0.0808, -0.2543,  0.2518,  ..., -0.2044,  0.1870,  0.0999],
         [-0.1664, -0.4297, -0.1914,  ..., -0.0775,  0.2719,  0.1007],
         [-0.


i=1710
Sanity Check!!

Start: 54720
End: 54752

Size of LastHidden Layer Size:torch.Size([32, 66, 768])
Pooler output size:torch.Size([32, 768])
            LastHidden Layer: tensor([[[-0.5811, -0.0481, -0.0222,  ..., -0.4722,  0.0084,  0.8056],
         [-0.7959, -0.3710, -0.2735,  ...,  0.0794,  0.3657, -0.7167],
         [-0.5715, -0.2303, -0.0551,  ..., -0.2161, -0.2024,  0.8684],
         ...,
         [-0.4412, -0.4625,  0.1441,  ...,  0.0023,  0.2790,  0.1590],
         [ 0.2225, -0.3506,  0.2633,  ..., -0.1192,  0.2365,  0.2484],
         [-0.3227, -0.6951,  0.0816,  ...,  0.0335,  0.3247, -0.0191]],

        [[-0.8617,  0.0148, -0.1581,  ..., -0.3013,  0.1721,  0.6167],
         [-0.5601,  0.0648, -0.2381,  ..., -0.0861, -0.0984, -0.5380],
         [-1.0396, -0.6927, -0.4200,  ...,  0.4761, -0.9501,  0.3220],
         ...,
         [-0.8117, -0.2397,  0.1281,  ..., -0.1736,  0.2829,  0.4800],
         [-0.7296, -0.1101, -0.0423,  ..., -0.0731,  0.2793,  0.4489],
         [-0.


i=1740
Sanity Check!!

Start: 55680
End: 55712

Size of LastHidden Layer Size:torch.Size([32, 50, 768])
Pooler output size:torch.Size([32, 768])
            LastHidden Layer: tensor([[[-1.0093, -0.1271, -0.2306,  ..., -0.5149,  0.1722,  1.0066],
         [-0.5753,  0.0229, -0.3885,  ...,  0.1471,  0.0945, -0.2974],
         [-0.4477, -0.3775, -0.6069,  ..., -0.3100, -0.5189,  0.6909],
         ...,
         [-1.0947, -0.1750,  0.0806,  ..., -0.2064,  0.2015,  0.8802],
         [-1.0987, -0.2349,  0.0762,  ..., -0.2285,  0.2484,  0.8604],
         [-0.9940, -0.1528,  0.0529,  ..., -0.3231,  0.0141,  0.7753]],

        [[-0.0446, -0.0382,  0.2009,  ..., -0.1000,  0.6883,  0.4686],
         [ 0.1332, -0.4041,  0.0233,  ...,  0.3974,  0.4271, -0.1816],
         [-0.0781, -0.6564,  0.8106,  ..., -0.3620, -0.2321,  0.0646],
         ...,
         [-0.1395, -0.1115,  0.1745,  ...,  0.4431,  0.4560, -0.4498],
         [-0.1269, -0.0154,  0.4235,  ...,  0.3803,  0.4507, -0.0520],
         [-0.